In [67]:
import sys, os
sys.path.append(os.path.abspath('..'))  # go up one level from notebooks/

In [68]:
import nfl_data_py as nfl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb as db
from utils.wp_predict import predict_wps

# ✅ Set display precision HERE
pd.set_option('display.precision', 6)
# Optional: to ensure everything aligns nicely
pd.options.display.float_format = '{:.6f}'.format

In [69]:
seasons = list(range(2015, 2025))
pbp = nfl.import_pbp_data(seasons)

print(pbp.shape)

2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
2024 done.
Downcasting floats.
(483605, 398)


In [70]:
pbp[['game_id','play_id','season','posteam','defteam','score_differential','wp']].head(100)

,game_id,play_id,season,posteam,defteam,score_differential,wp
0,2015_01_BAL_DEN,1.000000,2015,None,None,NaN,0.422024
1,2015_01_BAL_DEN,36.000000,2015,BAL,DEN,0.000000,0.422024
2,2015_01_BAL_DEN,51.000000,2015,BAL,DEN,0.000000,0.422024
3,2015_01_BAL_DEN,75.000000,2015,BAL,DEN,0.000000,0.420599
4,2015_01_BAL_DEN,96.000000,2015,BAL,DEN,0.000000,0.403295
...,...,...,...,...,...,...,...
95,2015_01_BAL_DEN,2075.000000,2015,DEN,BAL,-1.000000,0.486117
96,2015_01_BAL_DEN,2099.000000,2015,DEN,BAL,-1.000000,0.468685
97,2015_01_BAL_DEN,2125.000000,2015,BAL,DEN,1.000000,0.564145
98,2015_01_BAL_DEN,2149.000000,2015,BAL,DEN,1.000000,0.614148


In [71]:
con = db.connect()
con.register("plays", pbp)

df = con.sql("""
WITH tries AS (
  SELECT
    season,
    game_id,
    play_id,
    qtr,
    game_seconds_remaining,
    posteam AS team,                  -- team attempting the try (the leader)
    two_point_attempt,
    LOWER(COALESCE(two_point_conv_result,'')) AS two_point_conv_result,
    extra_point_attempt,
    LOWER(COALESCE(extra_point_result,''))     AS extra_point_result,
    wp,
    "desc"
  FROM plays
  WHERE qtr = 4
    AND score_differential = 7                 -- pre-try margin +7 for the try team
    AND (two_point_attempt = 1 OR extra_point_attempt = 1)
),
-- If a game has multiple qualifying tries (rare: penalties/retries), keep the first
dedup AS (
  SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY game_id ORDER BY play_id) AS rn
  FROM tries
)
SELECT
  season,
  game_id,
  qtr,
  team               AS leading_team,
  game_seconds_remaining,
  wp,
  (two_point_attempt = 1) AS went_for_two,
  CASE WHEN two_point_attempt = 1 THEN (two_point_conv_result = 'success') END AS two_point_success,
  extra_point_result,
  "desc"
FROM dedup
WHERE rn = 1
ORDER BY game_id
""").df()

In [72]:
sd7_Q4 = df.copy()

# manually correcting timestamps to align with PFR
sd7_Q4.loc[0, "game_seconds_remaining"] = 632
sd7_Q4.loc[1, "game_seconds_remaining"] = 594
sd7_Q4.loc[2, "game_seconds_remaining"] = 177
sd7_Q4.loc[3, "game_seconds_remaining"] = 559
sd7_Q4.loc[4, "game_seconds_remaining"] = 449
sd7_Q4.loc[5, "game_seconds_remaining"] = 698
sd7_Q4.loc[6, "game_seconds_remaining"] = 707
sd7_Q4.loc[7, "game_seconds_remaining"] = 568
sd7_Q4.loc[9, "game_seconds_remaining"] = 582

In [73]:
# Compute post-try WPs (leader perspective)
branches = sd7_Q4.apply(predict_wps, axis=1, base_margin=23)
sd7_Q4 = sd7_Q4.join(branches)
sd7_Q4.shape
sd7_Q4.head(50)

,season,game_id,qtr,leading_team,game_seconds_remaining,wp,went_for_two,two_point_success,extra_point_result,desc,wp_fail,wp_xp_good,wp_2pt_good
0,2015,2015_01_GB_CHI,4.000000,GB,632.000000,0.835824,False,<NA>,good,"2-M.Crosby extra point is GOOD, Center-61-B.Go...",0.996740,0.996740,0.997667
1,2015,2015_01_NO_ARI,4.000000,ARI,594.000000,0.857182,False,<NA>,good,"7-C.Catanzaro extra point is GOOD, Center-82-M...",0.997202,0.997202,0.998129
2,2015,2015_03_IND_TEN,4.000000,IND,177.000000,0.930985,False,<NA>,good,"4-A.Vinatieri extra point is GOOD, Center-45-M...",0.999694,0.999694,1.000000
3,2015,2015_03_TB_HOU,4.000000,HOU,559.000000,0.855037,False,<NA>,failed,"4-R.Bullock extra point is No Good, Wide Right...",0.997539,0.997539,0.998465
4,2015,2015_04_CLE_SD,4.000000,LAC,449.000000,0.868127,False,<NA>,good,"2-J.Lambo extra point is GOOD, Center-47-M.Win...",0.997803,0.997803,0.998730
5,2015,2015_09_TEN_NO,4.000000,NO,698.000000,0.839829,False,<NA>,good,"5-K.Forbath extra point is GOOD, Center-47-J.D...",0.996591,0.996591,0.997517
6,2015,2015_11_DEN_CHI,4.000000,DEN,707.000000,0.814417,False,<NA>,good,"8-B.McManus extra point is GOOD, Center-46-A.B...",0.996924,0.996924,0.997851
7,2015,2015_17_PHI_NYG,4.000000,PHI,568.000000,0.837419,False,<NA>,good,"6-C.Sturgis extra point is GOOD, Center-46-J.D...",0.997539,0.997539,0.998465
8,2016,2016_06_CAR_NO,4.000000,NO,362.000000,0.890421,False,<NA>,good,"3-W.Lutz extra point is GOOD, Center-47-J.Dres...",0.998543,0.998543,0.999470
9,2016,2016_08_NYJ_CLE,4.000000,NYJ,582.000000,0.832856,False,<NA>,good,"2-N.Folk extra point is GOOD, Center-46-T.Purd...",0.997539,0.997539,0.998465


In [77]:
p_xp, p_2pt = 0.95, 0.50

sd7_Q4['ev_xp']  = p_xp * sd7_Q4['wp_xp_good']  + (1 - p_xp) * sd7_Q4['wp_fail']
sd7_Q4['ev_2pt'] = p_2pt * sd7_Q4['wp_2pt_good'] + (1 - p_2pt) * sd7_Q4['wp_fail']
sd7_Q4['ev_diff_2pt_minus_xp'] = sd7_Q4['ev_2pt'] - sd7_Q4['ev_xp']

sd7_Q4.to_parquet("../data/sd7_Q4.parquet")